# Leaflet cluster map of talk locations

Assuming you are working in a Linux or Windows Subsystem for Linux environment, you may need to install some dependencies. Assuming a clean installation, the following will be needed:

```bash
sudo apt install jupyter
sudo apt install python3-pip
pip install python-frontmatter getorg --upgrade
```

After which you can run this from the `_talks/` directory, via:

```bash
 jupyter nbconvert --to notebook --execute talkmap.ipynb --output talkmap_out.ipynb
```
 
The `_talks/` directory contains `.md` files of all your talks. This scrapes the location YAML field from each `.md` file, geolocates it with `geopy/Nominatim`, and uses the `getorg` library to output data, HTML, and Javascript for a standalone cluster map.

In [2]:
# Start by installing the dependencies
!pip install python-frontmatter getorg --upgrade
import frontmatter
import glob
import getorg
from geopy import Nominatim
from geopy.exc import GeocoderTimedOut

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 1.8 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.7/432.7 kB 5.0 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 382.6/382.6 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 18.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 1.1 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.0
    Uninstalling typing_extensions-4.12.0:
      Successfully uninstalled typing_extensions-4.12.0
  Attempting uninstall: cryptography
    Found existing installation: cryptography 3.3.2
    Uninstalling cryptography-3.3.2:
      Successfully uninstalled cryptography-3.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour 

In [4]:
# Collect the Markdown files
g = glob.glob("_talks/*.md")

In [5]:
# Set the default timeout, in seconds
TIMEOUT = 5

# Prepare to geolocate
geocoder = Nominatim(user_agent="academicpages.github.io")
location_dict = {}
location = ""
permalink = ""
title = ""

In the event that this times out with an error, double check to make sure that the location is can be properly geolocated.

In [6]:
# Perform geolocation
for file in g:
    # Read the file
    data = frontmatter.load(file)
    data = data.to_dict()

    # Press on if the location is not present
    if 'location' not in data:
        continue

    # Prepare the description
    title = data['title'].strip()
    venue = data['venue'].strip()
    location = data['location'].strip()
    description = f"{title}<br />{venue}; {location}"

    # Geocode the location and report the status
    try:
        location_dict[description] = geocoder.geocode(location, timeout=TIMEOUT)
        print(description, location_dict[description])
    except ValueError as ex:
        print(f"Error: geocode failed on input {location} with message {ex}")
    except GeocoderTimedOut as ex:
        print(f"Error: geocode timed out on input {location} with message {ex}")
    except Exception as ex:
        print(f"An unhandled exception occurred while processing input {location} with message {ex}")

Seminar<br />Columbia University — Center for Theoretical Neuroscience; New York, NY, United States City of New York, New York, United States
Conference 'Analysis and Modeling of Connectomes'<br />Janelia Research Campus; Ashburn, VA, United States Ashburn, Loudoun County, Virginia, United States
Seminar<br />University of Geneva — Department of Basic Neurosciences; Geneva, Switzerland Genève, Schweiz/Suisse/Svizzera/Svizra
First Conference of the Italian Network for Computational Neuroscience (INCN)<br />Istituto Superiore di Sanità (ISS); Rome, Italy Roma, Roma Capitale, Lazio, Italia
Seminar<br />Bocconi University — Department of Decision Sciences; Milan, Italy Milano, Lombardia, Italia
Workshop 'Computational Aspects and Modeling of Biological Information'<br />Bocconi University; Milan, Italy Milano, Lombardia, Italia
Tutorial 'Modeling cortical networks dynamics'<br />CNS 2024; Natal, Brazil Natal, Região Geográfica Imediata de Natal, Região Geográfica Intermediária de Natal, Ri

In [7]:
# Save the map
m = getorg.orgmap.create_map_obj()
getorg.orgmap.output_html_cluster_map(location_dict, folder_name="talkmap", hashed_usernames=False)

'Written map to talkmap/'